In [23]:

import os
import collections

import numpy as np 

import pandas as pd 

import matplotlib.pyplot as plt 

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer as count_vect
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.externals import joblib

import nltk

% matplotlib inline


In [26]:
falsa = pd.read_csv(r'/home/roger/Documents/detector_fakenews/scraper/noticia_falsa/csv/boatos_org.csv')
verdadeira = pd.read_csv(r'/home/roger/Documents/detector_fakenews/scraper/noticia_verdadeira/csv/elpais.csv')


falsa = falsa.drop(columns=['quant', 'tema'])

#dropando itens para que ambos datasets tenham mesmo tamanho
#ate arranjar mais dados para o outro
verdadeira = verdadeira.drop(verdadeira.index[900:])

verdadeira = verdadeira.drop(columns=['quant'])

#Atribuindo a classe classificadora
# 1 para falso, 0 para verdadeiro

falsa['label'] = 1
verdadeira['label'] = 0

verdadeira.head(5)



,titulo_noticia,url,label
0,Indústria brasileira reage com melhora do comé...,https://brasil.elpais.com/brasil/2013/12/03/ec...,0
1,A bancarrota de Detroit deixa no ar as pensões...,https://brasil.elpais.com/brasil/2013/12/03/ec...,0
2,"PIB no Brasil cai 0,5% na leitura trimestral, ...",https://brasil.elpais.com/brasil/2013/12/03/ec...,0
3,O órgão supervisor europeu questiona o trabalh...,https://brasil.elpais.com/brasil/2013/12/02/ec...,0
4,"Vega Sicilia, a ilusão da escassez",https://brasil.elpais.com/brasil/2013/11/29/ec...,0


In [25]:
falsa.head(5)

,titulo_noticia,url,label
0,Neto de Chico Buarque fez versão de Águas de M...,www.boatos.org/entretenimento/neto-chico-buarq...,1
1,"Correios, em 2018, contrata em site “Correios ...",www.boatos.org/tecnologia/correios-2018-contra...,1
2,Caseiro do sítio de Atibaia diz que Lula tem c...,www.boatos.org/politica/caseiro-sitio-atibaia-...,1
3,Vídeo mostra rato tomando banho com sabonete e...,www.boatos.org/mundo/video-rato-tomando-banho....,1
4,Lutadora de vale tudo morre após enfrentar adv...,www.boatos.org/esporte/lutadora-morre-adversar...,1


In [19]:
dados = pd.concat([verdadeira,falsa])

#print(dados)

#Label encoder transforma valores categoricos em numericos

label_encoder = preprocessing.LabelEncoder()

y = label_encoder.fit_transform(dados.label.values)
x = dados.titulo_noticia.values

#print('printando x',x)

#Stratify = retorna mesma proporção
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=43, test_size=.33, stratify=y)

#print(collections.Counter(y_train))

#arq3 = "y_test.sav"
#joblib.dump(y_test, arq3)



collections.Counter(y_test)

#print(x_train.shape)
#print(x_test.shape)

#TF-IDF

#Baixando o 'bag of words'
#nltk.download('stopwords')

#Vetorizando palavras para numeros

pt_stopwords = set(nltk.corpus.stopwords.words('portuguese'))

tfidf = TfidfVectorizer(min_df = 3, strip_accents = 'unicode', max_features = 3000,
                        analyzer = 'word', token_pattern = '\w{1,}',
                        ngram_range = (1,3), sublinear_tf = 1, stop_words = pt_stopwords)

x_train_tfidf = tfidf.fit_transform(x_train)
x_test_tfidf = tfidf.transform(x_test)



In [108]:
#################################
#Testando com Regressao Logistica
#################################


#testando = tfidf.transform(texto_teste)

#print('TESTANDO', testando)

classificador = LogisticRegression()
classificador.fit(x_train_tfidf, y_train)

classificador.predict_proba(x_test_tfidf)

print('Acuracia do modelo LR: '
	,classificador.score(x_test_tfidf, y_test))

y_pred=classificador.predict(x_test_tfidf)


print(y_pred)

Acuracia do modelo LR:  0.8873949579831932
[0 1 1 1 0 0 0 0 1 0 0 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 0 1 1 1 0
 1 0 0 1 0 0 1 0 0 1 1 1 0 1 1 0 0 1 0 1 1 0 0 1 0 1 1 0 1 0 1 0 1 0 0 1 0
 1 1 1 0 1 0 0 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1 0 1 1 1 1 0 0 0 0 0 0 1 0 1 0
 0 0 1 1 0 1 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 1 0 1 0 1 0 1 1
 1 1 1 0 0 0 1 0 1 0 0 0 0 1 1 1 1 0 0 0 1 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 1
 1 0 1 0 0 1 0 0 1 1 0 1 0 0 1 1 0 1 0 0 1 1 1 0 1 1 1 0 1 1 1 1 0 0 1 0 0
 0 0 1 0 0 1 0 1 0 1 0 0 1 1 0 0 1 1 1 1 1 0 1 1 0 0 0 1 1 1 1 0 0 1 0 1 1
 0 0 1 1 1 1 0 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 1 0 0 0 1
 0 0 1 0 1 0 0 1 1 0 0 1 1 0 1 0 1 0 1 0 1 1 1 0 1 0 1 1 1 1 0 1 1 0 0 0 0
 1 1 1 0 1 0 1 0 0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 0 0 0 0 0 0 0 1 0
 0 1 0 0 0 0 0 1 0 1 1 1 0 1 1 0 0 0 1 1 0 1 0 1 1 1 0 1 1 1 0 1 0 1 1 1 0
 0 0 0 1 0 0 1 0 1 0 0 1 1 1 0 1 1 0 0 0 1 0 1 0 0 0 1 1 0 0 1 1 1 1 0 0 0
 1 0 0 0 1 1 1 0 1 1 1 1 0 0 0 0 0 0 1 0 0 1 0 1 1 0 1 1 

In [48]:
from sklearn import metrics

#cnf_matrix = metrics.confusion_matrix(y_test, y_pred)

In [122]:
texto_teste = ['Assalto programado para quem tem SKY, NET e GVT']
#texto_teste = ['Exames de Bolsonaro apontam pneumonia, diz boletim médico']


In [123]:

testando = tfidf.transform(texto_teste)

print(testando)


y_pred = classificador.predict(testando)

print('y', y_pred)

  (0, 403)	1.0
y [0]


In [112]:
print("Accuracy:",metrics.accuracy_score(texto_teste, y_pred))

Accuracy: 0.0


/home/roger/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:181: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


In [114]:
classificador.predict_proba(testando)

array([[0.59184222, 0.40815778]])